# <font color=blue>**TAREA 2**

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

from numpy import genfromtxt
from scipy.stats import multivariate_normal
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

## <font color=blue>**Parte 2.1 - Deteccion de Anomalias**

- Usando el dataset de estaturas(en google drive) aplicar detección de anomalías
    - Usar como training-set la primera pestaña “normales” (y=0)
    - La segunda pestaña “valtest(normales)” contiene datos normales(y=0):
        - Usar la mitad para cross-validation
        - Usar la mitad para testing
    - La tercera pestaña “valtest(anomalias)” contiene anomalías(y=1):
        - Usar la mitad para cross-validation 
        - Usar la mitad para testing
- Los datos de cross-validation deben ser usados para selección de hyper-parámetros(por ejemplo el umbral epsilon) y/o selección de transformaciones a aplicar a las variables.
- Usar los datos de testing para reportar las métricas de evaluación apropiadas
- Es permitido usar librerías estadísticas(por ejemplo scipy.stats) para cosas como:
    - Estimación de parámetros(función de densidad)
    - Cálculo de densidades y/o probabilidades con la función de densidad estimada


In [ ]:
EstaturasN = pd.read_excel('estaturas.xlsx','normales',dtype={'statura(metros)': float, 'Edad(años)': float})
EstaturasVT = pd.read_excel('estaturas.xlsx','valtest(normales)',dtype={'statura(metros)': float, 'Edad(años)': float})
EstaturasVA = pd.read_excel('estaturas.xlsx','valtest(anomalias)',dtype={'Estatura': float, 'Edad': float})
EstaturasVA

In [ ]:
#SPLIT DE DATOS
VT_Cross, VT_Test = train_test_split(EstaturasVT, test_size=0.5)
VA_Cross, VA_Test = train_test_split(EstaturasVA, test_size=0.5)

EstaturasC = np.vstack((np.array(VT_Cross),np.array(VA_Cross)))
EstaturasT = np.vstack((np.array(VT_Test),np.array(VA_Test)))
EstaturasL = np.concatenate((np.zeros(VT_Cross.shape[0]),np.ones(VA_Cross.shape[0])))

In [ ]:
def Calculo_MeanSD(Data):
    Mean = np.mean(Data, axis=0)
    StandardD = np.cov(Data.T)    
    return Mean, StandardD

In [ ]:
def Calculo_Densidad(Data,Mean, StandardD):
    return multivariate_normal(mean =Mean, cov= StandardD ).pdf(Data)

In [ ]:
def DeteccionAnomalia_Training(Data_Train, Data_Cross, Data_Label):
    
    m1, sd1 = Calculo_MeanSD(Data_Train)
    prob = Calculo_Densidad(Data_Train, m1,sd1)
    
    prob_a = Calculo_Densidad(Data_Cross, m1, sd1)
    
    best_epsilon = 0
    best_f1 = 0
    f = 0
    stepsize = (max(prob_a) - min(prob_a)) / 1000
    epsilons = np.arange(min(prob_a),max(prob_a),stepsize)
    for epsilon in np.nditer(epsilons):
        predictions = (prob_a < epsilon) 
        f = f1_score(Data_Label, predictions,average='binary')
        if f > best_f1:
            best_f1 = f
            best_epsilon = epsilon
    
    return best_f1, best_epsilon, m1, sd1


In [ ]:
f1, epsilon, m, sd = DeteccionAnomalia_Training(EstaturasN, EstaturasC, EstaturasL)

In [ ]:
def DeteccionAnomalia_Prediction(Data_Test, epsilon, m, sd):
    prob = Calculo_Densidad(Data_Test, m, sd)
            
    return prob<epsilon

In [ ]:
DeteccionAnomalia = DeteccionAnomalia_Prediction(EstaturasT, epsilon, m, sd)
print(DeteccionAnomalia)

In [ ]:
print("Anomalias")
print(EstaturasT[DeteccionAnomalia])

## <font color=blue>**Parte 2.2 - Reduccion de Dimensionalidad**

Usando el dataset de fashion MNIST realizar:
- PCA sin sklearn para reducir a 2 dimensiones.
    - Anotar la cantidad de varianza preservada.
- t-sne con sklearn para reducir a 2 dimensiones.
- Analizar ambas representaciones, comparar y concluir.
- Aplicar clustering con sklearn sobre la representación reducida. 
- Ya que este dataset si posee etiquetas “y” (tipo de prenda) analizar si los clusters encontrados tienden a agrupar el mismo tipo de prenda o prendas similares(por ejemplo se puede graficar cada tipo de diferente color)
- Agregar conclusiones finales. 

In [3]:
from tensorflow import keras

In [4]:
FMist = keras.datasets.fashion_mnist
(train_images, train_labels), (test_images, test_labels) = FMist.load_data()

In [5]:
np.unique(train_labels)

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=uint8)

In [6]:
train_images.shape

(60000, 28, 28)

In [7]:
# reshape dataset 
n, v1, v2 = train_images.shape
train_imagesRS = train_images.reshape((n,v1*v2))
train_imagesRS.shape

(60000, 784)

In [22]:
from sklearn import decomposition
ss = StandardScaler()
train_imagesRS_PCA = ss.fit_transform(train_imagesRS)
train_imagesRS_PCA .shape

(60000, 784)

**PCA**

In [23]:
#Calculo matriz de covarianza
CovM = np.cov(train_imagesRS_PCA.T)
#Calculo de eigen values
eigen_val, eigen_vec = np.linalg.eig(CovM)

In [24]:
total = sum(eigen_val)
varianza = np.array([])
for i in sorted(eigen_val, reverse=True):
    varianza=np.append(varianza,(i / total)) 
cum_varianza = np.cumsum(varianza)
cum_varianza

array([0.22083547, 0.3649818 , 0.41959356, 0.47048042, 0.51102929,
       0.5412244 , 0.56869979, 0.59183633, 0.6087627 , 0.62194313,
       0.63364417, 0.64326781, 0.65233222, 0.66094229, 0.66837236,
       0.67567561, 0.68229119, 0.68860634, 0.69482572, 0.70066855,
       0.70590244, 0.71101365, 0.71574898, 0.72024978, 0.72463682,
       0.72880767, 0.73279107, 0.73669652, 0.74047317, 0.74420843,
       0.74783961, 0.75135582, 0.75471624, 0.75802548, 0.76130592,
       0.76448097, 0.76753302, 0.77049963, 0.77334658, 0.77617726,
       0.77893312, 0.78161448, 0.78423821, 0.78678351, 0.78925873,
       0.79166115, 0.7940409 , 0.79632468, 0.79854539, 0.80070595,
       0.80281054, 0.80488254, 0.80691145, 0.80891752, 0.81090622,
       0.81284754, 0.81473285, 0.81658211, 0.81838592, 0.82015474,
       0.82190309, 0.82361652, 0.82530946, 0.82694574, 0.8285559 ,
       0.83015237, 0.83168996, 0.83319818, 0.83468497, 0.83614683,
       0.8376036 , 0.83903499, 0.84044351, 0.84183074, 0.84319

In [25]:
eigen_info = [(np.abs(eigen_val[i]), eigen_vec[:, i]) for i in range(len(eigen_val))]
eigen_info.sort(key=lambda k: k[0], reverse=True)

In [26]:
#extraccion de valores 
pca_data = np.hstack((eigen_info[0][1][:, np.newaxis], eigen_info[1][1][:, np.newaxis]))
print('2 Dimensiones (Aprox 0.36 varianza)', pca_data.shape)

2 Dimensiones (Aprox 0.36 varianza) (784, 2)


In [27]:
Z_PCA= np.matmul(pca_data.T,train_imagesRS_PCA.T)
Z_PCA.shape

(2, 60000)

**Con Sklearn**

In [14]:
pca = decomposition.PCA(n_components=2)
pca_data2 = pca.fit_transform(train_imagesRS)

In [39]:
pca_data2

array([[ -0.80117477,  20.86674867],
       [ 17.07420598,  -4.96851249],
       [ -9.60164085, -12.29610071],
       ...,
       [  9.37766441, -13.10585792],
       [ -9.64563637,  -7.09528763],
       [-21.31707928,  -1.74136728]])

**T-SNE**

In [28]:
from sklearn.manifold import TSNE

In [ ]:
#%%time
X_TSNE =  TSNE(n_components=2)
Z_TSNE = X_TSNE.fit_transform(train_imagesRS)

In [30]:
print(Z_TSNE.shape)

(60000, 2)


**KMeans**

In [31]:
from sklearn.cluster import KMeans

kmeans1 = KMeans(n_clusters=5, random_state=0).fit(Z_TSNE)
print(kmeans1.labels_)
print(kmeans1.cluster_centers_)

[4 1 4 ... 1 1 2]
[[-33.891685   17.817184 ]
 [  8.254221  -29.378826 ]
 [  6.9092145  30.345934 ]
 [ 33.893467   -0.7436141]
 [-25.188515  -15.906625 ]]


In [33]:
kmeans2 = KMeans(n_clusters=5, random_state=0).fit(pca_data2)
print(kmeans2.labels_)
print(kmeans2.cluster_centers_)

[2 4 0 ... 4 0 3]
[[ -3.09673523 -11.18185864]
 [ 19.17190918   5.66327313]
 [ -1.86770785  15.88380568]
 [-16.70428258   2.17163743]
 [ 10.46656426  -6.31500666]]


In [40]:
kmeans3 = KMeans(n_clusters=5, random_state=0).fit(Z_PCA.T)
print(kmeans3.labels_)
print(kmeans3.cluster_centers_)

[2 4 0 ... 4 0 3]
[[ -3.09673523  11.18185864]
 [ 19.17190918  -5.66327313]
 [ -1.86770785 -15.88380568]
 [-16.70428258  -2.17163743]
 [ 10.46656426   6.31500666]]
